# Advanced Querying Mongo

Importing libraries and setting up connection

In [62]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [2]:
# from pymongo import MongoClient
# client = MongoClient("") 
client.list_database_names()

['admin', 'config', 'dario', 'ironhack', 'local']

In [3]:
db = client.get_database("dario")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'dario')

In [4]:
db.list_collection_names()

['companies']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [5]:
# Your Code
name = list(db.companies.find({'name':'Babelgum'}, {'_id':0,'name':1}))

In [6]:
name

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [60]:
employees = db.companies.find({'number_of_employees': {'$gt': 5000}}, 
                        {'name':1, '_id':0}).sort('number_of_employees', 1).limit(20)

list(employees[:20])

[{'name': 'Nintendo'},
 {'name': 'Hexaware Technologies'},
 {'name': 'Facebook'},
 {'name': 'OpenText'},
 {'name': 'CPM Braxis'},
 {'name': 'LSI'},
 {'name': 'Microchip Technologies'},
 {'name': 'Mediaset'},
 {'name': 'Mindray Medical International'},
 {'name': 'Dentsu'},
 {'name': 'Atmel'},
 {'name': 'Tata Communications'},
 {'name': 'Baidu'},
 {'name': 'Acxiom'},
 {'name': 'Acxiom'},
 {'name': 'SRA International'},
 {'name': "Moody's"},
 {'name': 'ManTech'},
 {'name': 'Adobe Systems'},
 {'name': 'Broadcom'}]

In [8]:
# Your Code

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [59]:
founded = db.companies.find({'$and':[{'founded_year': {'$gte': 2000}},
                               {'founded_year': {'$lte': 2005}}]},
                      {'name':1, 'founded_year':1, '_id':0})

list(founded[:10])

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

In [10]:
# Your Code

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [58]:
valuation = db.companies.find({'$and':[{'ipo.valuation_amount': {'$gt': 100000000}},
                               {'founded_year': {'$lt': 2010}}]},
                      {'name':1, 'ipo.valuation_amount':1, '_id':0})

list(valuation[:10])          

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}}]

In [12]:
# Your Code

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [13]:
employees_2 = db.companies.find({'$and':[{'number_of_employees': {'$lt': 1000}},
                               {'founded_year': {'$lt': 2005}}]},\
                      {'name':1, '_id':0}).sort('number_of_employees', 1)

list(employees_2[:10]) 

[{'name': 'Skype'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Ticketmaster'},
 {'name': 'Fox Interactive Media'},
 {'name': 'stylediary'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Compete'},
 {'name': 'MindTouch'}]

### 6. All the companies that don't include the `partners` field.

In [68]:
partners = db.companies.count_documents({'partners':{'$exists': 1}})
print(partners)

18801


### 7. All the companies that have a null type of value on the `category_code` field.

In [17]:
nulos = db.companies.find({'category_code': None}, {'name':1, '_id':0})
list(nulos[:10])

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [57]:
employees_3 = db.companies.find({'$and':[{'number_of_employees': {'$lt': 1000}},
                               {'number_of_employees': {'$gte': 100}}]},
                      {'name':1, 'number_of_employees':1, '_id':0})
list(employees_3[:10])

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [56]:
price = db.companies.find({'ipo.valuation_amount': {'$gt': 0}},
                      {'name':1, 'ipo.valuation_amount':1, '_id':0}).sort('ipo', -1)

list(price[:10])  

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [64]:
ordenado = db.companies.find({'number_of_employees': {'$gte': 0}},
                      {'name':1, 'number_of_employees': 1, '_id':0})
                      .sort('number_of_employees', -1)

list(ordenado[:10]) 

IndentationError: unexpected indent (<ipython-input-64-5af640f4aa47>, line 3)

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [54]:
semestre = db.companies.find({'$and': [{'founded_month': {'$gte': 6}}, {'founded_month': {'$lte': 12}}]}
                      ,{'name':1, '_id':0}).limit(1000)

list(semestre[:10])

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Fox Interactive Media'},
 {'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Plaxo'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [53]:
todo = db.companies.find({'$and':[{'acquisition.price_amount': {'$gt': 10000000}},
                               {'founded_year': {'$lt': 2000}}]},
                      {'name':1, '_id':0})
list(todo[:10])

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [52]:
ordenado_2 = db.companies.find({'$and':[{'acquisition.price_amount': {'$gt': 10000000}},
                               {'founded_year': {'$lt': 2000}}]},
                      {'name':1, '_id':0})
list(ordenado_2[:10])

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [51]:
ordenar = db.companies.find({'number_of_employees': {'$gte': 0}},
                      {'name':1, 'founded_year':1, '_id':0}).limit(1000)

list(ordenar[:10]) 

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'AdventNet', 'founded_year': 1996},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Geni', 'founded_year': 2006},
 {'name': 'Fox Interactive Media', 'founded_year': 1979},
 {'name': 'Twitter', 'founded_year': 2006},
 {'name': 'Scribd', 'founded_year': 2007},
 {'name': 'eBay', 'founded_year': 1995}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [69]:
dias = db.companies.find({'founded_day': {'$lte': 7}},
                     {'name':1, '_id':0}).sort('acquisition.price_amount', -1)

list(dias[:10]) 

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [48]:
web = db.companies.find({'$and':[{'category_code': 'web'},{'number_of_employees': {'$gte':4000}}]},
                      {'name':1, '_id':0}).sort('number_of_employees', 1).limit(100)

list(web) 

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [49]:
euros = db.companies.find({'$and':[{'acquisition.price_amount': {'$gt': 10000000}},
                               {'acquisition.price_currency_code': 'EUR'}]},
                      {'name':1, '_id':0})
list(euros[:10])

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [61]:
trimestre = db.companies.find({'$and': [{'acquisition.acquired_month': {'$gte': 1}},
                                {'acquisition.acquired_month': {'$lte': 3}}]},
                      {'name':1, '_id':0, 'acquisition':1})

list(trimestre[:10])

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [46]:
fundados = db.companies.find({'$and': [{'founded_year': {'$gte': 2000}},
                                {'founded_year': {'$lte': 2010}},
                                {'acquisition.acquired_year': {'$gte': 2011}}]}
                      ,{'name':1, '_id':0})

list(fundados[:10])

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'},
 {'name': 'blogTV'},
 {'name': 'delicious'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# No lo he conseguido